# Wikipedia爬蟲練習
## 範例：練習是從Wikipedia中爬取文章。先定義一個搜尋的關鍵字，擷取該關鍵字詞的文章。

In [65]:
import requests
import re
from bs4 import BeautifulSoup

### 先定義一個我們想搜尋的字詞，並將它轉換成UTF-8編碼後的URL

In [66]:
input_keyword = "網路爬蟲"  # 這裡可以自己定義有興趣的關鍵字

utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
print("%s: %s" % (input_keyword, utf8_url[2:-1]))     # 擷取中間的編碼結果

# 組成Wiki關鍵字搜尋的網址格式
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
print(root_keyword_link)

網路爬蟲: %E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2
/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2


### 範例1：送出關鍵字請求後，爬取該關鍵字的文章內容

In [67]:
# 模擬封包的標頭
headers = {
    'authority': 'zh.wikipedia.org',
    'method': 'GET',
    'path': '/wiki/' + root_keyword_link,
    'scheme': 'https',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
    'accept-encoding': 'gzip, deflate, br',
    'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
    'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
    'dnt': '1',
    #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
    'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
}    

url = 'https://zh.wikipedia.org' + root_keyword_link  # 組合關鍵字查詢URL
print(url)
resp = requests.get(url, headers=headers)
resp.encoding = 'utf-8'

html = BeautifulSoup(resp.text, "lxml")
content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')

#
# 解析回傳資料，並萃取文章內容
#
for paragraph in content:
    print(paragraph)
    print(paragraph.get_text())
    break


https://zh.wikipedia.org/wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2
<p><b>網路爬蟲</b>（英語：<span lang="en">web crawler</span>），也叫<b>網路蜘蛛</b>（<span lang="en">spider</span>），是一種用來自動瀏覽<a href="/wiki/%E4%B8%87%E7%BB%B4%E7%BD%91" title="全球資訊網">全球資訊網</a>的<a href="/wiki/%E7%BD%91%E7%BB%9C%E6%9C%BA%E5%99%A8%E4%BA%BA" title="網路機器人">網路機器人</a>。其目的一般為編纂<span class="ilh-all" data-foreign-title="Web indexing" data-lang-code="en" data-lang-name="英語" data-orig-title="网络索引"><span class="ilh-page"><a class="new" href="/w/index.php?title=%E7%BD%91%E7%BB%9C%E7%B4%A2%E5%BC%95&amp;action=edit&amp;redlink=1" title="網路索引（頁面不存在）">網路索引</a></span><span class="noprint ilh-comment">（<span class="ilh-lang">英語</span><span class="ilh-colon">：</span><span class="ilh-link"><a class="extiw" href="https://en.wikipedia.org/wiki/Web_indexing" title="en:Web indexing"><span dir="auto" lang="en">Web indexing</span></a></span>）</span></span>。
</p>
網路爬蟲（英語：web crawler），也叫網路蜘蛛（spider），是一種用來自動瀏覽全球資訊網的網路機器人。其目的一般為編纂網路索引（英語：Web indexing）

### 範例2：從爬取的文章內容中，擷取出有外部連結的關鍵字。這些關鍵字在文章中是以藍色字體顯示，會連到外部的網頁，並解釋其內容。

In [68]:
for ext_link in content:
    a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
    #if len(a_tag) > 0:
    for link_string in a_tag:
            a_link = link_string["href"]       # 外部連結的網址
            a_keyword = link_string.get_text()  # 外部連結的中文名稱
            print("外部連結: [%s] %s" % (a_keyword, a_link))

外部連結: [全球資訊網] /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91
外部連結: [網路機器人] /wiki/%E7%BD%91%E7%BB%9C%E6%9C%BA%E5%99%A8%E4%BA%BA
外部連結: [網路] /wiki/%E7%BD%91%E7%BB%9C%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結: [搜尋引擎] /wiki/%E6%90%9C%E7%B4%A2%E5%BC%95%E6%93%8E
外部連結: [robots.txt] /wiki/Robots.txt
外部連結: [網站] /wiki/%E7%BD%91%E7%AB%99
外部連結: [超連結] /wiki/%E8%B6%85%E9%80%A3%E7%B5%90
外部連結: [HTML] /wiki/HTML
外部連結: [網頁] /wiki/%E7%B6%B2%E9%A0%81
外部連結: [網際網路] /wiki/%E4%BA%92%E8%81%94%E7%BD%91
外部連結: [伺服器] /wiki/%E6%9C%8D%E5%8A%A1%E5%99%A8
外部連結: [超文字傳輸協定] /wiki/%E8%B6%85%E6%96%87%E6%9C%AC%E5%82%B3%E8%BC%B8%E5%8D%94%E5%AE%9A


## 作業：接下來定義一個爬蟲函數，這個函數的主要工作為：
### (1) 爬取當前關鍵字的解釋，並存入檔案(因為文章內容太多會佔滿整個頁面，所以存程檔案，方便後續檢視)
### (2) 萃取出當前關鍵字所引用的外部連結，當作新的查詢關鍵字
### (3) 把第(2)擷取到的關鍵字當作新的關鍵字，回到第(1)步，爬取新的關鍵字解釋。

In [116]:
def WikiArticle(key_word_link, key_word, recursive):
    '''輸入網路爬蟲 
       第一層結果:
       遞迴層[0] - /wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2 (網路爬蟲)
    '''
    
    if (recursive <= max_recursive_depth):
        print("遞迴層[%d] - %s (%s)" % (recursive, key_word_link, key_word))
        
        # 模擬封包的標頭
        headers = {
            'authority': 'zh.wikipedia.org',
            'method': 'GET',
            'path': '/wiki/' + key_word_link,
            'scheme': 'https',
            'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3',
            'accept-encoding': 'gzip, deflate, br',
            'accept-language': 'zh-TW,zh;q=0.9,en-US;q=0.8,en;q=0.7,zh-CN;q=0.6',
            'cookie': 'GeoIP=TW:TPE:Taipei:25.05:121.53:v4; TBLkisOn=0; mwPhp7Seed=8b8; WMF-Last-Access-Global=04-Jun-2019; WMF-Last-Access=04-Jun-2019',
            'dnt': '1',
            #'if-modified-since': 'Tue, 04 Jun 2019 12:03:22 GMT',
            'referer': 'https://zh.wikipedia.org/wiki/Wikipedia:%E9%A6%96%E9%A1%B5',
            'upgrade-insecure-requests': '1',
            'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36'
        }    

        url = 'https://zh.wikipedia.org' + key_word_link  # 組合關鍵字查詢URL
        resp = requests.get(url, headers=headers)
        resp.encoding = 'utf-8'

        html = BeautifulSoup(resp.text, "lxml")
        content = html.find(name='div', attrs={'id':'mw-content-text'}).find_all(name='p')
        
        #
        # Part 1: 請參考範例1，爬取當前關鍵字的文章內容。
        #         因為內容太多，我們把它寫入檔案，並以關鍵字作為檔案名稱，以便稍後查閱內容。
        #         請先建立一個名為"WikiArticle"的資料夾，爬取到的文章內容會放在這個資料夾底下。
        #
        
        # 根據需求引入正確的 Library
        from urllib.request import urlretrieve
        import os
        # 建立資料夾WikiArticle
        output_dir = 'WikiArticle'

        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        '''    
        try:
            os.makedirs( './WikiArticle', exist_ok=True )
            #urlretrieve("https://www.w3.org/TR/PNG/iso_8859-1.txt","./Data/Homework.txt")
            #print('建立Wiki資料夾')
        except:
            print('發生錯誤！')
        '''
        #整理內容準備存檔
        paragraphstrings=''
        icnt = 0
        for paragraph in content:
            paragraphstrings = paragraphstrings + paragraph.get_text()
            icnt = icnt+1
            if icnt > 5 :
                break
        #print(paragraphstrings)
        
        # 將搜尋內容，覆寫到 關鍵字檔案
        #filename="./WikiArticle/"+key_word+'.txt'
        #print(filename)
        # 建立關鍵字檔名的檔案，並存入資料
        output_file = '{outdir}/{outfile}.txt'.format(
            outdir=output_dir, outfile=key_word.replace('/', '_'))
        
        try:
            with open(output_file, "w",encoding="utf-8") as fh:
                fh.write(paragraphstrings)
        except IOError as err:
            print (err)
        finally:
            fh.close()
        # Part 2: 請參考範例2，萃取出本篇文章中所延伸引用的外部連結，並儲存在external_link_dict
        #
        external_link_dict = dict({})
        for ext_link in content:
            a_tag = ext_link.find_all('a', href=re.compile("^(/wiki/)((?!;)\S)*$"))
            for link_string in a_tag:
                a_link = link_string["href"]       # 外部連結的網址
                a_keyword = link_string.get_text()  # 外部連結的中文名稱
                #[全球資訊網] /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91
                external_link_dict[ a_keyword ] = a_link
                #print("外部連結: [%s] %s" % (a_keyword, a_link))
            #print(external_link_dict.items().keys())
               
                   
        #
        # Part 3: 將Part 2所收集的外部連結，當作新的關鍵字，繼續迭代深入爬蟲
        #
        if (len(external_link_dict) > 0):
            
            recursive = recursive + 1  # 遞迴深度加1
            
            for k, v in external_link_dict.items():
                #print(k,' v=',v,' recursive=',recursive)
                WikiArticle(v, k, recursive)  # 再次呼叫同樣的函數，執行同樣的流程
        

In [117]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2
input_keyword = "網路爬蟲"  # 這裡可以自己定義有興趣的關鍵字
utf8_url = repr(input_keyword.encode('UTF-8')).upper()  # 編碼成UTF-8並轉成大寫字元
utf8_url = utf8_url.replace("\\X", "%")                 # 用 '%' 取代 '\X' 
root_keyword_link = '/wiki/' + utf8_url[2:-1:1]
#print(root_keyword_link)

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2 (網路爬蟲)
遞迴層[1] - /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91 (全球資訊網)
遞迴層[2] - /wiki/%E5%9B%A0%E7%89%B9%E7%BD%91 (網際網路)
遞迴層[2] - /wiki/%E8%B6%85%E6%96%87%E6%9C%AC (超文字)
遞迴層[2] - /wiki/%E8%92%82%E5%A7%86%C2%B7%E4%BC%AF%E7%BA%B3%E6%96%AF-%E6%9D%8E (提姆·柏內茲-李)
遞迴層[2] - /wiki/CERN (CERN)
遞迴層[2] - /wiki/%E7%B6%B2%E9%A0%81%E7%80%8F%E8%A6%BD%E5%99%A8 (網頁瀏覽器)
遞迴層[2] - /wiki/%E8%B3%87%E8%A8%8A%E6%99%82%E4%BB%A3 (資訊時代)
遞迴層[2] - /wiki/%E7%BD%91%E9%A1%B5 (網頁)
遞迴層[2] - /wiki/%E6%A0%BC%E5%BC%8F%E5%8C%96%E6%96%87%E6%9C%AC (格式化)
遞迴層[2] - /wiki/%E8%B6%85%E6%96%87%E4%BB%B6%E6%A8%99%E7%A4%BA%E8%AA%9E%E8%A8%80 (超文件標示語言)
遞迴層[2] - /wiki/%E5%9C%96%E7%89%87 (圖片)
遞迴層[2] - /wiki/%E5%BD%B1%E7%89%87 (影片)
遞迴層[2] - /wiki/%E8%81%B2%E9%9F%B3 (聲音)
遞迴層[2] - /wiki/%E5%A4%9A%E5%AA%92%E9%AB%94 (多媒體)
遞迴層[2] - /wiki/ENQUIRE (ENQUIRE)
遞迴層[2] - /wiki/%E6%95%B0%E6%8D%AE%E5%BA%93 (資料庫)
遞迴層[2] - /wiki/%E8%AF%AD%E4%B9%89%E7%BD%91 (語意網)
遞迴層[2] - /wiki/%E7%BD%97%E4%BC%AF%E7%89%B9%C2%B7%E5%8D%A

遞迴層[1] - /wiki/%E7%BD%91%E7%AB%99 (網站)
遞迴層[2] - /wiki/%E8%90%AC%E7%B6%AD%E7%B6%B2 (全球資訊網)
遞迴層[2] - /wiki/HTML (HTML)
遞迴層[2] - /wiki/%E7%B6%B2%E9%A0%81 (網頁)
遞迴層[2] - /wiki/%E7%BD%91%E7%BB%9C%E6%9C%8D%E5%8A%A1 (網路服務)
遞迴層[2] - /wiki/%E7%B6%B2%E9%A0%81%E7%80%8F%E8%A6%BD%E5%99%A8 (網頁瀏覽器)
遞迴層[2] - /wiki/%E6%8B%9B%E8%81%98 (招聘)
遞迴層[2] - /wiki/%E5%80%8B%E4%BA%BA%E4%B8%BB%E9%A0%81 (個人主頁)
遞迴層[2] - /wiki/%E6%96%B0%E8%81%9E (新聞)
遞迴層[2] - /wiki/%E6%97%85%E9%81%8A (旅遊)
遞迴層[2] - /wiki/%E5%A8%9B%E6%A8%82 (娛樂)
遞迴層[2] - /wiki/%E7%B6%93%E6%BF%9F (經濟)
遞迴層[2] - /wiki/%E5%9C%96%E5%83%8F (圖像)
遞迴層[2] - /wiki/%E8%81%B2%E9%9F%B3 (聲音)
遞迴層[2] - /wiki/%E5%8B%95%E7%95%AB (動畫)
遞迴層[2] - /wiki/%E5%BD%B1%E7%89%87 (影片)
遞迴層[2] - /wiki/3D (3D)
遞迴層[2] - /wiki/%E5%8B%95%E6%85%8B%E7%B6%B2%E9%A0%81 (動態網頁)
遞迴層[2] - /wiki/%E9%9B%BB%E5%AD%90%E9%83%B5%E4%BB%B6 (電子郵件)
遞迴層[2] - /wiki/%E8%92%82%E5%A7%86%C2%B7%E4%BC%AF%E7%BA%B3%E6%96%AF-%E6%9D%8E (提姆·柏內茲-李)
遞迴層[2] - /wiki/CERN (CERN)
遞迴層[2] - /wiki/%E5%80%8B%E4%BA%BA%E7%B6%B2%E7%AB%9

遞迴層[2] - /wiki/%E7%BE%8E%E5%9B%BD%E5%9B%BD%E9%98%B2%E9%83%A8 (美國國防部)
遞迴層[2] - /wiki/%E5%9C%8B%E9%98%B2%E9%AB%98%E7%AD%89%E7%A0%94%E7%A9%B6%E8%A8%88%E5%8A%83%E7%BD%B2 (高等研究計劃署)
遞迴層[2] - /wiki/%E5%86%B7%E6%88%98 (冷戰)
遞迴層[2] - /wiki/ARPANET (ARPA網)
遞迴層[2] - /wiki/%E8%8B%B1%E5%9B%BD (英國)
遞迴層[2] - /wiki/%E6%8C%AA%E5%A8%81 (挪威)
遞迴層[2] - /wiki/%E8%AE%A1%E7%AE%97%E6%9C%BA (電腦)
遞迴層[2] - /wiki/%E7%BD%97%E4%BC%AF%E7%89%B9%C2%B7%E5%8D%A1%E6%81%A9 (羅伯特·卡恩)
遞迴層[2] - /wiki/%E6%96%87%E9%A0%93%C2%B7%E7%91%9F%E5%A4%AB (文頓·瑟夫)
遞迴層[2] - /wiki/TCP/IP%E5%8D%8F%E8%AE%AE (TCP/IP協定)
遞迴層[2] - /wiki/%E5%9B%BE%E7%81%B5%E5%A5%96 (圖靈獎)
遞迴層[2] - /wiki/%E7%BD%91%E7%BB%9C%E6%8E%A7%E5%88%B6%E5%8D%8F%E8%AE%AE (NCP)
遞迴層[2] - /wiki/%E7%BE%8E%E5%9B%BD%E5%9B%BD%E5%AE%B6%E7%A7%91%E5%AD%A6%E5%9F%BA%E9%87%91%E4%BC%9A (美國國家科學基金會)
遞迴層[2] - /wiki/Usenet (Usenet)
遞迴層[2] - /wiki/Bitnet (Bitnet)
遞迴層[2] - /wiki/X.25 (X.25)
遞迴層[2] - /wiki/%E8%92%82%E5%A7%86%C2%B7%E4%BC%AF%E7%BA%B3%E6%96%AF-%E6%9D%8E (提姆·柏內茲-李)
遞迴層[2] - /wiki/%E7%91%9E

遞迴層[2] - /wiki/%E4%BD%BF%E7%94%A8%E8%80%85%E7%B6%93%E9%A9%97 (用戶感受到的)
遞迴層[2] - /wiki/Gopher_(%E7%BD%91%E7%BB%9C%E5%8D%8F%E8%AE%AE) (Gopher)
遞迴層[2] - /wiki/SPDY (SPDY)
遞迴層[2] - /wiki/Google (Google)


### 執行前個步驟定義好的爬蟲主程式

In [14]:
# 定義爬取的遞迴深度。深度不要訂太深，否則會爬很久。
max_recursive_depth = 2

WikiArticle(root_keyword_link, input_keyword, 0)

遞迴層[0] - /wiki/%E7%B6%B2%E8%B7%AF%E7%88%AC%E8%9F%B2 (網路爬蟲)
遞迴層[1] - /wiki/%E4%B8%87%E7%BB%B4%E7%BD%91 (全球資訊網)
遞迴層[2] - /wiki/%E4%BA%92%E8%81%94%E7%BD%91 (網際網路)
遞迴層[2] - /wiki/%E8%B6%85%E6%96%87%E6%9C%AC (超文字)
遞迴層[2] - /wiki/%E8%92%82%E5%A7%86%C2%B7%E4%BC%AF%E7%BA%B3%E6%96%AF-%E6%9D%8E (提姆·柏內茲-李)
遞迴層[2] - /wiki/CERN (CERN)
遞迴層[2] - /wiki/%E7%B6%B2%E9%A0%81%E7%80%8F%E8%A6%BD%E5%99%A8 (網頁瀏覽器)
遞迴層[2] - /wiki/%E8%B3%87%E8%A8%8A%E6%99%82%E4%BB%A3 (資訊時代)
遞迴層[2] - /wiki/%E7%B6%B2%E9%A0%81 (網頁)
遞迴層[2] - /wiki/%E6%A0%BC%E5%BC%8F%E5%8C%96%E6%96%87%E6%9C%AC (格式化)
遞迴層[2] - /wiki/%E8%B6%85%E6%96%87%E4%BB%B6%E6%A8%99%E7%A4%BA%E8%AA%9E%E8%A8%80 (超文件標示語言)
遞迴層[2] - /wiki/%E5%9C%96%E7%89%87 (圖片)
遞迴層[2] - /wiki/%E5%BD%B1%E7%89%87 (影片)
遞迴層[2] - /wiki/%E8%81%B2%E9%9F%B3 (聲音)
遞迴層[2] - /wiki/%E5%A4%9A%E5%AA%92%E9%AB%94 (多媒體)
遞迴層[2] - /wiki/%E7%B6%B2%E9%9A%9B%E7%B6%B2%E8%B7%AF (網際網路)
遞迴層[2] - /wiki/ENQUIRE (ENQUIRE)
遞迴層[2] - /wiki/%E6%95%B0%E6%8D%AE%E5%BA%93 (資料庫)
遞迴層[2] - /wiki/%E8%AF%AD%E4%B9%89%E7%BD%91 (語意

遞迴層[2] - /wiki/%E5%A8%9B%E6%A8%82 (娛樂)
遞迴層[2] - /wiki/%E7%B6%93%E6%BF%9F (經濟)
遞迴層[2] - /wiki/%E5%9C%96%E5%83%8F (圖像)
遞迴層[2] - /wiki/%E8%81%B2%E9%9F%B3 (聲音)
遞迴層[2] - /wiki/%E5%8B%95%E7%95%AB (動畫)
遞迴層[2] - /wiki/%E5%BD%B1%E7%89%87 (影片)
遞迴層[2] - /wiki/3D (3D)
遞迴層[2] - /wiki/%E5%8B%95%E6%85%8B%E7%B6%B2%E9%A0%81 (動態網頁)
遞迴層[2] - /wiki/%E9%9B%BB%E5%AD%90%E9%83%B5%E4%BB%B6 (電子郵件)
遞迴層[2] - /wiki/%E8%92%82%E5%A7%86%C2%B7%E4%BC%AF%E7%BA%B3%E6%96%AF-%E6%9D%8E (提姆·柏內茲-李)
遞迴層[2] - /wiki/CERN (CERN)
遞迴層[2] - /wiki/%E5%80%8B%E4%BA%BA%E7%B6%B2%E7%AB%99 (個人網站)
遞迴層[2] - /wiki/%E7%94%B5%E5%AD%90%E6%94%BF%E5%8A%A1 (政府網站)
遞迴層[2] - /wiki/%E9%9D%9E%E7%87%9F%E5%88%A9%E7%B5%84%E7%B9%94 (非營利組織)
遞迴層[2] - /wiki/%E8%B6%85%E9%80%A3%E7%B5%90 (超連結)
遞迴層[2] - /wiki/%E8%B6%85%E6%96%87%E4%BB%B6%E6%A8%99%E7%A4%BA%E8%AA%9E%E8%A8%80 (超文件標示語言)
遞迴層[2] - /wiki/%E4%BD%BF%E7%94%A8%E8%80%85%E4%BB%A3%E7%90%86 (使用者代理)
遞迴層[2] - /wiki/%E8%BB%9F%E9%AB%94 (軟體)
遞迴層[2] - /wiki/%E9%9B%BB%E8%85%A6 (電腦)
遞迴層[2] - /wiki/%E7%AD%86%E8%A8%98%E5%9

遞迴層[2] - /wiki/%E5%9B%BE%E7%81%B5%E5%A5%96 (圖靈獎)
遞迴層[2] - /wiki/%E7%BD%91%E7%BB%9C%E6%8E%A7%E5%88%B6%E5%8D%8F%E8%AE%AE (NCP)
遞迴層[2] - /wiki/%E7%BE%8E%E5%9B%BD%E5%9B%BD%E5%AE%B6%E7%A7%91%E5%AD%A6%E5%9F%BA%E9%87%91%E4%BC%9A (美國國家科學基金會)
遞迴層[2] - /wiki/Usenet (Usenet)
遞迴層[2] - /wiki/Bitnet (Bitnet)
遞迴層[2] - /wiki/X.25 (X.25)
遞迴層[2] - /wiki/%E8%92%82%E5%A7%86%C2%B7%E4%BC%AF%E7%BA%B3%E6%96%AF-%E6%9D%8E (提姆·柏內茲-李)
遞迴層[2] - /wiki/%E7%91%9E%E5%A3%AB (瑞士)
遞迴層[2] - /wiki/%E6%AD%90%E6%B4%B2%E6%A0%B8%E5%AD%90%E7%A0%94%E7%A9%B6%E7%B5%84%E7%B9%94 (歐洲核子研究組織)
遞迴層[2] - /wiki/HTML (HTML)
遞迴層[2] - /wiki/HTTP (HTTP)
遞迴層[2] - /wiki/FidoNet (FidoNet)
遞迴層[2] - /wiki/ARPANET (ARPA網)
遞迴層[2] - /wiki/%E7%BE%8E%E8%81%94%E7%A4%BE (美聯社)
遞迴層[2] - /wiki/%E7%BA%BD%E7%BA%A6%E6%97%B6%E6%8A%A5 (紐約時報)
遞迴層[2] - /wiki/%E5%AF%AC%E5%B8%B6 (寬頻)
遞迴層[2] - /wiki/%E7%AA%84%E9%A0%BB (窄頻)
遞迴層[2] - /wiki/%E5%9E%83%E5%9C%BE%E9%82%AE%E4%BB%B6 (垃圾郵件)
遞迴層[2] - /wiki/%E8%A0%95%E8%99%AB%E7%97%85%E6%AF%92 (蠕蟲病毒)
遞迴層[2] - /wiki/%E6%83%A1%E6%8